
# SRK/T Parameter‑Fitting Notebook 🇮🇹

Questo notebook calcola **MAE** e **RMSE** della formula SRK/T “pura” sui dati
del foglio *Data* e poi ottimizza l’indice cheratometrico `n_k` e,
facoltativamente, l’indice corneale `n_e`, per ridurre l’errore.


In [1]:
import pandas as pd

# se notebook e file xlsx sono nello stesso folder
xls_path = 'FacoDMEK.xlsx'        # <-- percorso relativo

# apri il file e mostra i nomi dei fogli per sicurezza
xls = pd.ExcelFile(xls_path)
print('Fogli disponibili:', xls.sheet_names)

# carica il foglio “Data” (rispetta maiuscole/minuscole)
df = pd.read_excel(xls, sheet_name='Data')
print(f'Dataset shape: {df.shape}')
df.head()


Fogli disponibili: ['Data', 'Excluded']
Dataset shape: (96, 29)


,ID,Patient,Eye,Sex,Birthdate,PreOP Diagnosis,Date of Surgery,Age,PostOP BCVA,PostOP Spherical Equivalent,...,CCT,Keratometric Km,Keratometric Ks,Keratometric Kf,Anterior Km,Anterior Ks,Anterior Kf,Posterior Km,Posterior Ks,Posterior Kf
0,1,maggioni giovanni,OS,M,1948-06-01,Fuchs,2020-06-09,72,1,-3.875,...,873,45.00,46.3,43.7,50.15,51.6,48.7,-5.70,-7.3,-4.1
1,2,alberghina andrea,OD,M,1967-06-01,Fuchs,2021-02-03,53,0.6,-3.125,...,699,47.20,48.0,46.4,52.60,53.5,51.7,-6.60,-6.8,-6.4
2,3,munteanu lenuta,OD,F,1961-06-01,Fuchs,2022-02-03,60,0.9,-3.000,...,511,45.45,45.7,45.2,50.60,50.9,50.3,-6.35,-6.5,-6.2
3,4,ferrando adriana,OD,F,1949-06-01,Fuchs,2023-05-10,73,1,-2.250,...,665,44.65,44.8,44.5,49.70,49.9,49.5,-5.90,-5.9,-5.9
4,5,bianucci daniele,OD,M,1982-06-01,Fuchs,2023-11-22,41,0.8,-2.250,...,857,46.15,47.7,44.6,51.45,53.2,49.7,-5.60,-5.9,-5.3


In [2]:

import numpy as np

def srkt_predict(row, n_k=1.3375, n_e=1.333, n_a=1.336,
                 lcor_coeff=(3.446, 1.716, -0.0237),
                 cw_coeff=(-5.41, 0.58412, 0.098),
                 rethick_coeff=(0.65696, -0.02029),
                 offset_base=3.336, V=12.0):
    # --- inputs ---
    AL = row['Bio-AL']
    K1 = row['Bio-Ks']
    K2 = row['Bio-Kf']
    K = (K1 + K2) / 2.0  # mean K
    r = 337.5 / K        # anterior corneal radius
    
    # --- LCOR ---
    if AL <= 24.4:
        LCOR = AL
    else:
        a0, a1, a2 = lcor_coeff
        LCOR = a0 + a1 * AL + a2 * AL ** 2
    
    # --- Cw & H ---
    c0, c1, c2 = cw_coeff
    Cw = c0 + c1 * LCOR + c2 * K
    H  = r - np.sqrt(max(r**2 - (Cw**2)/4, 0))
    
    # --- ACD_est ---
    Aconst = row['A-Constant']
    ACD_const = 0.62467 * Aconst - 68.747
    offset = ACD_const - offset_base
    ACD_est = H + offset
    
    # --- RETHICK & LOPT ---
    r0, r1 = rethick_coeff
    RETHICK = r0 + r1 * AL
    LOPT = AL + RETHICK
    
    # --- thin‑lens equation ---
    ncml = n_e - 1.0
    num = 1000 * n_a * (n_a * r - ncml * LOPT)
    den = (V * (n_a * r - ncml * LOPT) + LOPT * r)
    pred_SE = num / den -               row['IOL Power'] * (LOPT - ACD_est) * (n_a * r - ncml * ACD_est) /               (n_a * (V * (n_a * r - ncml * ACD_est) + ACD_est * r))
    return pred_SE


## Baseline SRK/T

In [3]:

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

df['SE_pred'] = df.apply(srkt_predict, axis=1)
residuals = df['PostOP Spherical Equivalent'] - df['SE_pred']
baseline_mae  = mean_absolute_error(df['PostOP Spherical Equivalent'], df['SE_pred'])
baseline_rmse = np.sqrt(mean_squared_error(df['PostOP Spherical Equivalent'], df['SE_pred']))

print(f'Baseline MAE  : {baseline_mae:.3f} D')
print(f'Baseline RMSE : {baseline_rmse:.3f} D')


Baseline MAE  : 2.567 D
Baseline RMSE : 2.939 D


## Ottimizzazione n_k e n_e

In [4]:

from scipy.optimize import minimize

def objective(theta):
    n_k, n_e = theta
    preds = df.apply(srkt_predict, axis=1, args=(n_k, n_e))
    return mean_squared_error(df['PostOP Spherical Equivalent'], preds)

# bounds: (1.330,1.345) for both
res = minimize(objective, x0=[1.3375, 1.333], bounds=[(1.33,1.345),(1.33,1.337)])
best_nk, best_ne = res.x
print(f'Optimal n_k: {best_nk:.5f}, Optimal n_e: {best_ne:.5f}')

df['SE_opt'] = df.apply(srkt_predict, axis=1, args=(best_nk,best_ne))
opt_mae  = mean_absolute_error(df['PostOP Spherical Equivalent'], df['SE_opt'])
opt_rmse = np.sqrt(mean_squared_error(df['PostOP Spherical Equivalent'], df['SE_opt']))
print(f'Optimized MAE : {opt_mae:.3f} D')
print(f'Optimized RMSE: {opt_rmse:.3f} D')


Optimal n_k: 1.33750, Optimal n_e: 1.33000
Optimized MAE : 2.461 D
Optimized RMSE: 2.894 D
